# Clean UK schools data

This notebook cleans some UK schools data sets and joins them with other data sources as deprivation data.

Main datasets:
* [gov.uk Comparing School Website](https://www.compare-school-performance.service.gov.uk/)
* [English indices of deprivation 2015](http://imd-by-postcode.opendatacommunities.org/)

In [1]:
import pandas as pd
import numpy as np

### Utility functions

Some utility function to cast data to proper data types. 

In [2]:
def is_int(value):
        try:
            int(value)
            return True
        except ValueError:
            return False
        
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

to_float = lambda x: float(x if is_float(x) else np.nan)
to_int = lambda x: int(x) if is_int(x) else np.nan

### Read and clean SPINE data

Load and clean SPINE data. For each school the following fields are loaded:
 * school URN
 * la
 * school name
 * locality
 * town
 * postcode

In [3]:
def clean_spine(file_path):
    
    def la_codes(file_path):
        la_codes = pd.read_csv(file_path, usecols=['LEA', 'LA Name'])
        la_codes.columns = ['la', 'la name']
        return la_codes
    
    la_codes = la_codes('/project/uk-schools-clustering/data/meta/la_and_region_codes_meta.csv')
    spine = pd.read_csv(
        file_path,
        usecols=['URN', 'LA', 'SCHNAME', 'LOCALITY', 'TOWN', 'POSTCODE'],
        dtype={
            'URN': 'object'
        }
    )
    spine['POSTCODE'] = spine['POSTCODE'].str.replace(' ', '')
    spine.columns = ['urn', 'la', 'name', 'locality', 'town', 'postcode']
    return pd.merge(spine, la_codes, on=['la']).drop_duplicates('urn')

In [4]:
clean_spine('/project/uk-schools-clustering/data/raw/2016-2017_england_spine.csv').sample(5)

,urn,la,name,locality,town,postcode,la name
7646,106428,359,Lowton West Primary School,Lowton,Warrington,WA32ED,Wigan
8721,107873,383,Ninelands Primary School,Garforth,Leeds,LS251NT,Leeds
9093,139501,384,Carleton Community High School A Specialist Sc...,Carleton,Pontefract,WF83NW,Wakefield
6018,104481,340,St Margaret Mary's Catholic Infant School,NaN,Liverpool,L140JG,Knowsley
23687,123366,893,Gobowen Primary School,Gobowen,Oswestry,SY113LD,Shropshire


### Read and clean census data

Load and clean some schools census data, including:
* total number of pupils on roll
* number of girls on roll
* number of boys on roll
* number of pupils which english is not first language
* number of pupils which english is first language
* number of pupils on free meals

In [5]:
def clean_census(file_path):
    census = pd.read_csv(
        file_path, usecols=['URN', 'NOR', 'NORG', 'NORB', 'NUMEAL', 'NUMENGFL', 'PNUMFSM'],
        converters={
            'NOR': to_int,
            'NORG': to_int,
            'NORB': to_int,
            'NUMEAL': to_int,
            'NUMENGFL': to_int,
            'PNUMFSM': to_float
        }
    )
    census['on free meal'] = (census['NORG']*census['PNUMFSM']) / 100
    census['on free meal'] = round(census['on free meal'])
    census.drop(inplace=True, columns=['PNUMFSM'])
    census.columns = ['urn', 'total pupils on roll', 
                      'girls on roll', 'boys on roll', 
                      'english not first language', 'english first language', 'on free meal']
    return census.drop_duplicates('urn')
    

In [6]:
clean_census('/project/uk-schools-clustering/data/raw/2016-2017_england_census.csv')

,urn,total pupils on roll,girls on roll,boys on roll,english not first language,english first language,on free meal
0,100000,276.0,136.0,140.0,105.0,72.0,16.0
1,100001,727.0,727.0,0.0,0.0,0.0,0.0
2,100002,248.0,103.0,145.0,0.0,0.0,0.0
3,100003,928.0,0.0,928.0,0.0,0.0,0.0
4,100008,396.0,198.0,198.0,275.0,22.0,56.0
5,100009,446.0,226.0,220.0,236.0,117.0,58.0
6,100010,390.0,177.0,213.0,213.0,117.0,55.0
7,100011,437.0,207.0,230.0,70.0,274.0,36.0
8,100012,359.0,160.0,199.0,229.0,67.0,62.0
9,100013,217.0,112.0,105.0,151.0,13.0,39.0


### Read and clean workforce data

Load and clean school workforce data, such as:
* Total number of teachers (headcount)
* Mean gross fulltime teacher salary

In [7]:
def clean_workforce(file_path):
    clean_salary = lambda x : x.replace('£', '').replace(',','.')
    workforce = pd.read_csv(
        file_path,
        usecols=['URN', 'Total Number of Teachers (Headcount)', 'Mean Gross FTE Salary of All Teachers'],
        dtype={'URN': object},
        converters={
            'Total Number of Teachers (Headcount)': to_int,
            'Mean Gross FTE Salary of All Teachers': lambda x: to_float(clean_salary(x))
        }
    )
    workforce.columns = ['urn', 'teacher headcount', 'mean salary fte']
    return workforce
    

In [8]:
clean_workforce('/project/uk-schools-clustering/data/raw/2016-2017_england_swf.csv')

,urn,teacher headcount,mean salary fte
0,100000,20.0,46.053
1,136807,6.0,45.589
2,139837,6.0,49.225
3,140686,0.0,NaN
4,100008,22.0,44.640
5,100009,27.0,45.328
6,100010,26.0,39.253
7,100011,28.0,42.305
8,100012,22.0,46.462
9,100013,13.0,43.912


### Read and clean spending data

Load school spending data:
* Total school income per pupil
* Total school expenditure per pupil

In [9]:
def clean_spending(file_path):
    clean_value = lambda x : x.replace(',','.')
    to_float = lambda x: float(clean_value(x) if is_float(clean_value(x)) else np.nan)
    spending = pd.read_csv(
        file_path,
        usecols=['URN', 'TOTALINCOME', 'TOTALEXPENDITURE'],
        dtype={
            'URN': 'object'
        },
        converters={
            'TOTALINCOME': lambda x : to_float(x),
            'TOTALEXPENDITURE': lambda x : to_float(x)
        }
    )
    spending.columns = ['urn', 'total income pp', 'total expenditure pp']
    return spending

In [10]:
clean_spending('/project/uk-schools-clustering/data/raw/2016-2017_england_cfr.csv')

,urn,total income pp,total expenditure pp
0,100000,8.176,8.319
1,100005,12.646,12.280
2,100006,40.968,40.968
3,100007,84.399,81.137
4,100008,6.869,6.660
5,100009,6.444,6.293
6,100010,7.191,6.644
7,100011,5.630,5.700
8,100012,7.526,7.911
9,100013,7.162,7.108


### Read and clean deprivation data

Load [UK deprivation data](https://www.gov.uk/government/statistics/english-indices-of-deprivation-2015):
* Income score
* Employment score
* IDACI score

In [11]:
def clean_deprivation(file_path):
    deprivation = pd.read_csv(
        file_path,
        usecols=['Postcode', 'Income Score', 'Employment Score', 'IDACI Score'],
        converters={
            'Postcode' : lambda s : s.replace(' ', ''),
            'Income Score': lambda x : to_float(x),
            'Employment Score': lambda x : to_float(x),
            'IDACI Score': lambda x : to_float(x)
        }
    )
    deprivation.columns = ['postcode', 'income score', 'empl score', 'idaci score']
    return deprivation

In [12]:
clean_deprivation('/project/uk-schools-clustering/data/raw/deprivation-by-postcode-2015.csv')

,postcode,income score,empl score,idaci score
0,EC3A5DE,0.014,0.011,0.051
1,EC2Y8BB,0.012,0.017,0.006
2,EC4M9AD,0.014,0.011,0.051
3,EC4V3AL,0.014,0.011,0.051
4,EC1M6EA,0.029,0.031,0.018
5,WC1H9EG,0.267,0.216,0.311
6,NW61QL,0.088,0.058,0.128
7,NW19AL,0.180,0.118,0.309
8,N195DH,0.225,0.175,0.284
9,NW54AX,0.367,0.247,0.507


### Read and clean key stage 2 final data

Load final key stage 2 data. Including:
* math, writing, reading and grammar average scaled scores
* percentage of pupils meeting math, writing, reading and grammar standards

In [13]:
def clean_k2final(file_path):
    def clean_percent(percent_str):
        percent_candidate = percent_str.replace('%', '')
        return to_float(percent_candidate) / 100

    k2final = pd.read_csv(
        file_path,
        usecols=['URN', 'PTREAD_EXP', 
                 'PTMAT_EXP', 'PTGPS_EXP', 'PTWRITTA_EXP', 
                 'READ_AVERAGE', 'GPS_AVERAGE', 'MAT_AVERAGE'
        ],
        converters={
            'PTREAD_EXP' : clean_percent,
            'PTMAT_EXP' : clean_percent,
            'PTGPS_EXP' : clean_percent,
            'PTWRITTA_EXP' : clean_percent,
            'READ_AVERAGE' : to_int,
            'GPS_AVERAGE' : to_int,
            'MAT_AVERAGE' : to_int
        }
    )
    k2final.rename(columns={
        'URN':'urn', 
        'PTREAD_EXP': 'perc pupils meeting reading standard',
        'PTMAT_EXP': 'perc pupils meeting math standard',
        'PTGPS_EXP': 'perc pupils meeting grammar standard',
        'PTWRITTA_EXP': 'perc pupils meeting writing standard',
        'READ_AVERAGE': 'avg reading scaled score',
        'GPS_AVERAGE': 'avg grammar scaled score',
        'MAT_AVERAGE': 'avg math scaled score'
    }, inplace=True)
    return k2final

In [14]:
clean_k2final('/project/uk-schools-clustering/data/raw/2016-2017_england_ks2final.csv')

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2856: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,urn,perc pupils meeting reading standard,avg reading scaled score,perc pupils meeting grammar standard,avg grammar scaled score,perc pupils meeting math standard,avg math scaled score,perc pupils meeting writing standard
0,110705,0.54,99.0,0.63,101.0,0.67,100.0,0.75
1,117736,0.67,104.0,0.78,106.0,0.67,101.0,0.56
2,122638,0.86,106.0,0.90,110.0,0.71,104.0,0.86
3,141550,0.88,111.0,0.75,105.0,0.88,103.0,0.88
4,142999,0.73,106.0,0.73,106.0,0.78,106.0,0.80
5,106190,0.63,102.0,0.87,107.0,0.67,103.0,0.87
6,120819,0.75,104.0,0.56,99.0,0.75,102.0,0.81
7,120602,0.73,102.0,0.55,100.0,0.73,102.0,0.55
8,135197,0.87,105.0,0.87,105.0,0.91,105.0,0.91
9,105667,0.75,106.0,0.85,108.0,0.85,105.0,0.85


### Merge datasets

This function, loads all different datasets, merges them together and eventually saves them on disk.

In [86]:
def get_data(save_to = None, columns = None):
    spine = clean_spine('/project/uk-schools-clustering/data/raw/2016-2017_england_spine.csv')
    census = clean_census('/project/uk-schools-clustering/data/raw/2016-2017_england_census.csv')
    workforce = clean_workforce('/project/uk-schools-clustering/data/raw/2016-2017_england_swf.csv')
    spending = clean_spending('/project/uk-schools-clustering/data/raw/2016-2017_england_cfr.csv')
    deprivation = clean_deprivation('/project/uk-schools-clustering/data/raw/deprivation-by-postcode-2015.csv')
    k2final = clean_k2final('/project/uk-schools-clustering/data/raw/2016-2017_england_ks2final.csv')
    result = pd.merge(spine, census, on=['urn'])
    result = pd.merge(result, deprivation, on=['postcode'])
    result = pd.merge(result, workforce, on=['urn'])
    result = pd.merge(result, spending, on=['urn'])
    result = pd.merge(result, k2final, on=['urn'])
    result.dropna(axis=0, subset=[
        'total income pp',
        'idaci score',
        'mean salary fte',
        'perc pupils meeting reading standard',
        'perc pupils meeting grammar standard',
        'perc pupils meeting math standard',
        'avg reading scaled score',
        'avg grammar scaled score',
        'avg math scaled score'
    ], how='any', inplace=True)
#     result.dropna(axis=0, how='all', inplace=True)
    if columns is None:
        columns_to_select = result.columns
    else:
        columns_to_select = columns
    if save_to is not None:
        result[columns_to_select].to_csv(save_to, index=False)
    return result[columns_to_select]

In [87]:
get_data(
    '/project/uk-schools-clustering/data/derived/2016-2017_england.csv',
    columns=['urn', 'name', 'english first language', 'girls on roll',
             'english not first language','total income pp', 'total pupils on roll', 'on free meal',
             'idaci score', 'teacher headcount','boys on roll', 'mean salary fte', 'total expenditure pp',
             'income score', 'empl score', 'perc pupils meeting reading standard',
             'perc pupils meeting math standard', 'perc pupils meeting grammar standard', 'perc pupils meeting writing standard',
             'avg reading scaled score','avg grammar scaled score','avg math scaled score']
)

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,urn,name,english first language,girls on roll,english not first language,total income pp,total pupils on roll,on free meal,idaci score,teacher headcount,...,total expenditure pp,income score,empl score,perc pupils meeting reading standard,perc pupils meeting math standard,perc pupils meeting grammar standard,perc pupils meeting writing standard,avg reading scaled score,avg grammar scaled score,avg math scaled score
0,100000,Sir John Cass's Foundation Primary School,72.0,136.0,105.0,8.176,276.0,16.0,0.051,20.0,...,8.319,0.014,0.011,0.88,0.92,0.92,0.92,107.0,109.0,107.0
1,100009,Beckford Primary School,117.0,226.0,236.0,6.444,446.0,58.0,0.128,27.0,...,6.293,0.088,0.058,0.64,0.75,0.76,0.75,101.0,105.0,104.0
2,100010,Brecknock Primary School,117.0,177.0,213.0,7.191,390.0,55.0,0.309,26.0,...,6.644,0.180,0.118,0.78,0.93,0.87,0.82,106.0,112.0,110.0
3,100011,Brookfield Primary School,274.0,207.0,70.0,5.630,437.0,36.0,0.284,28.0,...,5.700,0.225,0.175,0.82,0.78,0.69,0.67,106.0,106.0,105.0
4,100012,Carlton Primary School,67.0,160.0,229.0,7.526,359.0,62.0,0.507,22.0,...,7.911,0.367,0.247,0.85,0.91,0.88,0.79,106.0,108.0,107.0
5,100014,Fleet Primary School,82.0,136.0,92.0,6.581,228.0,29.0,0.192,17.0,...,6.856,0.121,0.083,0.81,0.81,0.78,0.70,107.0,105.0,107.0
6,100018,Netley Primary School & Centre for Autism,41.0,189.0,301.0,8.492,439.0,70.0,0.281,26.0,...,8.408,0.168,0.100,0.68,0.77,0.77,0.77,103.0,107.0,104.0
7,100019,New End Primary School,124.0,197.0,220.0,5.539,438.0,25.0,0.090,24.0,...,5.649,0.120,0.072,0.80,0.78,0.85,0.80,107.0,109.0,106.0
8,100023,Torriano Primary School,168.0,205.0,187.0,7.180,448.0,55.0,0.412,26.0,...,7.090,0.191,0.112,0.93,0.93,0.92,0.86,111.0,112.0,109.0
9,100025,Gospel Oak Primary School,187.0,238.0,170.0,6.360,468.0,58.0,0.193,23.0,...,6.276,0.113,0.070,0.75,0.85,0.93,0.88,106.0,110.0,106.0
